In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random
import numpy as np
import scipy.stats as scistats
import stabilityUtils as su
from scipy import stats
from tqdm import tqdm
DEFAULT_R2 = 0.36
DEFAULT_SLOPE = 0.048

In [ ]:
EXPERIMENTS_10_31_2025 = "../src/build/csv/experiment-31-10-2025.csv"

In [ ]:
DF_EXPERIMENT_PICKLE = "./experiment-31-10.df.pickle"
df_experiment_boostraped = su.load_or_compute(DF_EXPERIMENT_PICKLE,su.load_n_compute_dataframe,EXPERIMENTS_10_31_2025)

In [ ]:
df_experiment_boostraped.columns

In [ ]:
def analyze_entropy_stability(entropy_values: np.ndarray, 
                               max_angle_deg: float = 0.048, 
                               min_r2: float = 0.36):
    if len(entropy_values) < 2:
        return False, float(0), float(0)

    x = np.arange(len(entropy_values))
    y = np.array(entropy_values)
    slope, intercept, r_value, _, _ = stats.linregress(x, y)
    r2 = r_value ** 2

    slope_angle_deg = np.degrees(np.arctan(slope))

    slope_check_passed = abs(slope_angle_deg) <= max_angle_deg
    r2_check_passed = r2 >= min_r2
    stable = bool(slope_check_passed and r2_check_passed)
    
    return stable,float(slope_angle_deg),float(r2)

In [ ]:
def compute_binned_entropy(measurements, bin_resolution_ms: float = 0.0005) -> float:
    if len(measurements) == 0:
        return 0.0
    epsilon = bin_resolution_ms * 2.0
    binned_keys = np.round(measurements / epsilon) * epsilon
    unique_bins, counts = np.unique(binned_keys, return_counts=True)
    total_samples = len(measurements)
    probabilities = counts / total_samples
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy

In [ ]:
df_narrow = df_experiment_boostraped[df_experiment_boostraped["version"] == "ABBilateral"]
df_narrow =df_narrow[df_narrow["repetitions"] == 1000]
df_narrow =df_narrow[df_narrow["warmups"] == 5]
df_narrow =df_narrow[df_narrow["work_size"] == 1]
df_narrow =df_narrow[df_narrow["flush_l2"] == 1]
df_narrow =df_narrow[df_narrow["blocking"] == 0]

In [ ]:
def compute_entropy_stable(row,batch_entropy_size=2,batch_linear_size=50):
    nb_repetitions = row.repetitions
    
    entropies = []
    r2s = []
    stables = []
    slopes = []
    count = 1
    for i in range(batch_entropy_size,nb_repetitions,batch_entropy_size):
        entropies.append(compute_binned_entropy(row.repetitions_duration[:i]))
        if i >= (batch_linear_size*count):
            stable, slope, r2 = analyze_entropy_stability(entropies)
            stables.append(stable)
            r2s.append(r2)
            slopes.append(slope)
            count+=1
    
    return entropies[-1],r2s[-1],stables[-1],slopes[-1]

In [ ]:
def compute_entropy_graph(row,batch_entropy_size=2,batch_linear_size=50):
    nb_repetitions = row.repetitions
    
    entropies = []
    r2s = []
    stables = []
    slopes = []
    for i in range(batch_entropy_size,nb_repetitions,batch_entropy_size):
        entropies.append(compute_binned_entropy(row.repetitions_duration[:i]))
        if i % batch_linear_size == 0:
            stable, slope, r2 = analyze_entropy_stability(entropies)
            stables.append(stable)
            r2s.append(r2)
            slopes.append(slope)
    
    return entropies,r2s,stables,slopes

In [ ]:
def compute_entropy_criterion(dataframe,batch_entropy_size=2,batch_linear_size=5):
    df_e = []
    df_e = dataframe.copy()
   
    df_e['slope'] = None
    df_e['r2'] = None
    df_e['entropy'] = None
    df_e['stable_entropy'] = None

    for idx in tqdm(df_e.index):
        entropies,r2s,stables,slopes = compute_entropy_graph(df_e.loc[idx],batch_entropy_size=batch_entropy_size,batch_linear_size=batch_linear_size)
        df_e.at[idx,'slope'] = slopes
        df_e.at[idx,'r2'] = r2s
        df_e.at[idx,'entropy'] = entropies
        df_e.at[idx,'stable_entropy'] = stables
    return df_e

In [ ]:
def compute_entropy_choice(dataframe,batch_entropy_size=2,batch_linear_size=5):
    df_e = []
    df_e = dataframe.copy()
   
    df_e['slope'] = None
    df_e['r2'] = None
    df_e['entropy'] = None
    df_e['stable_entropy'] = None

    for idx in tqdm(df_e.index):
        entropy,r2,stable,slope = compute_entropy_stable(df_e.loc[idx],batch_entropy_size=batch_entropy_size,batch_linear_size=batch_linear_size)
        df_e.at[idx,'slope'] = slope
        df_e.at[idx,'r2'] = r2
        df_e.at[idx,'entropy'] = entropy
        df_e.at[idx,'stable_entropy'] = stable
    return df_e


In [ ]:
df_experiment_boostraped

In [ ]:
df_exp_boot_entropy = compute_entropy_choice(df_experiment_boostraped)

In [ ]:
df_exp_boot_entropy

In [ ]:
def plot_evolution_thing(dataframe,x,stable,line_val=None,min=None):
    plt.plot(dataframe["repetitions"], dataframe[x], color='black')          # continuous line
    plt.scatter(dataframe["repetitions"], dataframe[x], c=np.where(dataframe[stable], 'green', 'blue'))  # markers colored by stable
    if line_val:
        plt.axhline(y=line_val, color='red', linestyle='--', label=f"{x}-min")
        if min == True:
            plt.fill_between(dataframe["repetitions"], -1, line_val, color='green', alpha=0.2)
        elif min == False:
            plt.fill_between(dataframe["repetitions"], line_val, max(dataframe[x] + [line_val])*1.1, color='green', alpha=0.2)
    
    plt.xlabel("nbr of repetitions")
    plt.ylabel(x)
    plt.title(f"Evolution of {x}")


In [ ]:
def plot_things_single_version(dataframe,version="ReferenceBilateral",work_size=1):
    dataframe = dataframe[dataframe["version"]==version]
    dataframe = dataframe[dataframe["warmups"]==5]
    dataframe = dataframe[dataframe["blocking"]==0]
    dataframe = dataframe[dataframe["work_size"]==work_size]
    dataframe = dataframe[dataframe["flush_l2"]==1]
    fig, axs = plt.subplots(1, 3, figsize=(10, 3), constrained_layout=True)
    plt.sca(axs[0])             
    plot_evolution_thing(dataframe,"entropy","stable_entropy")
    plt.sca(axs[1])             
    plot_evolution_thing(dataframe,"r2","stable_entropy",DEFAULT_R2,min=False)
    plt.sca(axs[2])             
    plot_evolution_thing(dataframe,"slope","stable_entropy",DEFAULT_SLOPE,min=True)
    
    fig.suptitle(f" Kernel {version}, work_size {work_size}", fontsize=16)

    return fig

In [ ]:
for wz in df_exp_boot_entropy["work_size"].unique():
    plot_things_single_version(df_exp_boot_entropy,work_size=wz)

In [ ]:
dataframe = df_exp_boot_entropy[df_exp_boot_entropy["blocking"]==0]
dataframe = dataframe[dataframe["work_size"]==2]
dataframe = dataframe[dataframe["flush_l2"]==1]
f = su.plot_compare_repetitions_multiple_version(df_exp_boot_entropy)

In [ ]:
def plot_entropy(entropie_a,r2_a,slope_a,nb_repetitions,btch_entropy_sz,btch_linear_sz,r2_min=DEFAULT_R2,slope_min=DEFAULT_SLOPE):
    fig, axes = plt.subplots(1, 3, figsize=(10, 3), sharex=True)

    axes[0].plot(
        range(btch_entropy_sz,
              nb_repetitions, btch_entropy_sz),
        entropie_a,
    )
    axes[0].set_title("Entropies")
    axes[0].set_ylabel("Entropy")

    axes[1].plot(
        range(btch_linear_sz, nb_repetitions, btch_linear_sz),
        r2_a,
    )
    axes[1].set_title("R² values")
    axes[1].set_ylabel("R²")
    axes[1].axhline(y=r2_min, color='red', linestyle='--', label='r2_min')
    x_linear = range(btch_linear_sz, nb_repetitions, btch_linear_sz)
    axes[1].fill_between(x_linear, r2_min, max(r2_a + [r2_min])*1.1, color='green', alpha=0.2)

    axes[1].legend()

    axes[2].plot(
        range(btch_linear_sz, nb_repetitions, btch_linear_sz),
        slope_a,
    )
    axes[2].set_title("Slopes")
    axes[2].set_xlabel("Repetitions")
    axes[2].set_ylabel("Slope")
    axes[2].axhline(y=slope_min, color='red', linestyle='--', label='slope_min')
    axes[2].fill_between(x_linear, -1, slope_min, color='green', alpha=0.2)
    axes[2].legend()

    plt.tight_layout()
    plt.show()